## Data Mining Techniques
### COVID-19 data
Kimberley Boersma (2572145), Neil Mizzi (2674737), Selma Muhammad (Stud no)

In [1]:
# Imports
import os
import pandas as pd
import csv
import kaggle

# other imports
import numpy as np 
# import matplotlib.pyplot as plt 
# import matplotlib.colors as mcolors
# import random
# import math
# import time
# from sklearn.linear_model import LinearRegression, BayesianRidge
# from sklearn.model_selection import RandomizedSearchCV, train_test_split
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.svm import SVR
# from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import date, datetime
from dateutil.parser import parse
# import us
# import operator 
# plt.style.use('fivethirtyeight')
# %matplotlib inline 

# Covid Tracking Dataset (w/ hospitalised data)

Source: https://covidtracking.com/

## Step 1: Load and Clean the Data

In [2]:
all_cases = pd.read_csv('https://covidtracking.com/api/v1/states/daily.csv')

# Delete unecessary rows
for row in ['negative', 'pending', 'hash', 'negativeIncrease', 'totalTestResults', 'totalTestResultsIncrease', 'dateChecked', 'fips', 'inIcuCumulative', 'onVentilatorCumulative', 'total', 'posNeg', 'deathIncrease', 'hospitalizedIncrease', 'positiveIncrease']:
    del all_cases[row]

# TODO missing values
#      Do we get avg or missing values, or predict them?
#      See https://developerzen.com/data-mining-handling-missing-values-the-database-bd2241882e72

for i, row in all_cases.iterrows():
    # Set Dates
    s = str(row['date'])
    all_cases.at[i, 'date'] = date(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8]))
    

# Missing death figures means no death reports yet
# These are set to 0
for i, row in all_cases.iterrows():
    if np.isnan(row['death']):
        all_cases.at[i, 'death'] = 0

## Missing values: Retrieving from other datasets or through merging columns (or both)

The following will be done:
- **Active Cases**: Retrieved from $active = pos-dead-recovered$
- **Beds per State**: Retrieved from External Datasets

In [3]:
# TODO Replace active cases with JHU and/or regression model (Selma)
all_cases['active'] = all_cases['positive'] - all_cases['recovered'] - all_cases['death']
# change location of 'active' column
cols = list(all_cases)
cols.insert(3, cols.pop(cols.index('active')))
all_cases = all_cases.loc[:, cols]

In [4]:
# Load datasets for US population and Hospital beds per 1000
us_population = pd.read_csv('data/us_population.csv')
hosp_beds = pd.read_csv('data/hospital_beds.csv')
state_abbrev = pd.read_csv('data/us_state_names.csv')

# add state abbreviations to us_population and hospital beds dataframe
for state in state_abbrev['State'].tolist():
    # store state abbreviation in variable
    abbrev = state_abbrev.loc[state_abbrev['State'] == state, 'Abbreviation'].tolist()[0]
    # add abbrev to new column 'Abbreviation' in us_population df
    us_population.loc[us_population['State'] == state, 'Abbreviation'] = abbrev
    # add abbrev to new column in hosp_beds df
    hosp_beds.loc[hosp_beds['Location'] == state, 'Abbreviation'] = abbrev
    
# change order of columns of us_population
cols = list(us_population)
cols.insert(2, cols.pop(cols.index('Abbreviation')))
us_population = us_population.loc[:, cols]

# drop unnecessary columns of us_population
us_population = us_population.drop(columns=['rank', 'Growth', 'Pop2018', 'Pop2010', 'growthSince2010', 'Percent', 'density'])

# drop unnecessary columns of hosp_beds
hosp_beds = hosp_beds.drop(columns=['Location', 'State/Local Government', 'Non-Profit', 'For-Profit'])

# change order of columns of hosp_beds
cols = list(hosp_beds)
cols.insert(0, cols.pop(cols.index('Abbreviation')))
hosp_beds = hosp_beds.loc[:, cols]

In [5]:
us_population.head()

,State,Abbreviation,Pop
0,Alabama,AL,4908621
1,Alaska,AK,734002
2,Arizona,AZ,7378494
3,Arkansas,AR,3038999
4,California,CA,39937489


In [6]:
hosp_beds.head()

,Abbreviation,Total
0,NaN,2.4
1,AL,3.1
2,AK,2.2
3,AZ,1.9
4,AR,3.2


In [7]:
# filter out non-existing states like 'AS'
all_cases = all_cases[all_cases['state'].isin(state_abbrev['Abbreviation'].tolist())]

In [8]:
# see what filtered dataframe looks like
all_cases.head()

,date,state,positive,active,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,onVentilatorCurrently,recovered,death,hospitalized
0,2020-04-26,AK,341.0,115.0,14.0,NaN,NaN,NaN,217.0,9.0,NaN
1,2020-04-26,AL,6270.0,NaN,NaN,845.0,NaN,NaN,NaN,216.0,845.0
2,2020-04-26,AR,2941.0,1907.0,104.0,291.0,NaN,25.0,985.0,49.0,291.0
4,2020-04-26,AZ,6526.0,4868.0,671.0,1037.0,308.0,200.0,1383.0,275.0,1037.0
5,2020-04-26,CA,42164.0,NaN,4928.0,NaN,1473.0,NaN,NaN,1710.0,NaN


In [9]:
# check which states have 0 positive cases
all_cases.loc[all_cases['positive'] == 0]

,date,state,positive,active,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,onVentilatorCurrently,recovered,death,hospitalized
2294,2020-03-17,WV,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2350,2020-03-16,WV,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2401,2020-03-15,WV,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2452,2020-03-14,WV,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2467,2020-03-13,ID,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2859,2020-03-05,NE,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2864,2020-03-05,OH,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2867,2020-03-05,SC,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2870,2020-03-05,VA,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [32]:
# Split dataframes by date
df_split_by_date = dict(tuple(all_cases.groupby('date')))

# Split dataframes by state
df_split_by_state = dict(tuple(all_cases.groupby('state')))

In [11]:
# merge dataframes us_population and all_cases
df_merge_uspop = all_cases.merge(us_population, how='left', left_on='state', right_on='Abbreviation')
df_merge_uspop = df_merge_uspop.drop(columns=['Abbreviation'])
df_merge_uspop = df_merge_uspop.rename(columns={'Pop': 'population'})

# change location of 'population' column
cols = list(df_merge_uspop)
cols.insert(2, cols.pop(cols.index('population')))
df_merge_uspop = df_merge_uspop.loc[:, cols]

# merge dataframes hosp_beds and df_merge_uspop
df_merge_hosp = df_merge_uspop.merge(hosp_beds, how='left', left_on='state', right_on='Abbreviation')
df_merge_hosp = df_merge_hosp.drop(columns=['Abbreviation'])
all_cases = df_merge_hosp.rename(columns={'Total': 'bedsPerThousand'})

In [12]:
all_cases.head()

,date,state,population,positive,active,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,onVentilatorCurrently,recovered,death,hospitalized,State,bedsPerThousand
0,2020-04-26,AK,734002,341.0,115.0,14.0,NaN,NaN,NaN,217.0,9.0,NaN,Alaska,2.2
1,2020-04-26,AL,4908621,6270.0,NaN,NaN,845.0,NaN,NaN,NaN,216.0,845.0,Alabama,3.1
2,2020-04-26,AR,3038999,2941.0,1907.0,104.0,291.0,NaN,25.0,985.0,49.0,291.0,Arkansas,3.2
3,2020-04-26,AZ,7378494,6526.0,4868.0,671.0,1037.0,308.0,200.0,1383.0,275.0,1037.0,Arizona,1.9
4,2020-04-26,CA,39937489,42164.0,NaN,4928.0,NaN,1473.0,NaN,NaN,1710.0,NaN,California,1.8


In [13]:
# Calculate the total beds, and add the column
all_cases['total_beds'] = all_cases['population'] / 1000 * all_cases['bedsPerThousand']
all_cases

,date,state,population,positive,active,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,onVentilatorCurrently,recovered,death,hospitalized,State,bedsPerThousand,total_beds
0,2020-04-26,AK,734002,341.0,115.0,14.0,NaN,NaN,NaN,217.0,9.0,NaN,Alaska,2.2,1614.8044
1,2020-04-26,AL,4908621,6270.0,NaN,NaN,845.0,NaN,NaN,NaN,216.0,845.0,Alabama,3.1,15216.7251
2,2020-04-26,AR,3038999,2941.0,1907.0,104.0,291.0,NaN,25.0,985.0,49.0,291.0,Arkansas,3.2,9724.7968
3,2020-04-26,AZ,7378494,6526.0,4868.0,671.0,1037.0,308.0,200.0,1383.0,275.0,1037.0,Arizona,1.9,14019.1386
4,2020-04-26,CA,39937489,42164.0,NaN,4928.0,NaN,1473.0,NaN,NaN,1710.0,NaN,California,1.8,71887.4802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2722,2020-01-26,WA,7797095,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Washington,1.7,13255.0615
2723,2020-01-25,WA,7797095,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Washington,1.7,13255.0615
2724,2020-01-24,WA,7797095,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Washington,1.7,13255.0615
2725,2020-01-23,WA,7797095,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Washington,1.7,13255.0615


In [14]:
# change abbreviations to state names
all_cases = all_cases.rename(columns={'state': 'abbrev'})
all_cases = all_cases.rename(columns={'State': 'state'})

In [15]:
# change location of 'state' column
cols = list(all_cases)
cols.insert(1, cols.pop(cols.index('state')))
all_cases = all_cases.loc[:, cols]

In [16]:
all_cases

,date,state,abbrev,population,positive,active,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,onVentilatorCurrently,recovered,death,hospitalized,bedsPerThousand,total_beds
0,2020-04-26,Alaska,AK,734002,341.0,115.0,14.0,NaN,NaN,NaN,217.0,9.0,NaN,2.2,1614.8044
1,2020-04-26,Alabama,AL,4908621,6270.0,NaN,NaN,845.0,NaN,NaN,NaN,216.0,845.0,3.1,15216.7251
2,2020-04-26,Arkansas,AR,3038999,2941.0,1907.0,104.0,291.0,NaN,25.0,985.0,49.0,291.0,3.2,9724.7968
3,2020-04-26,Arizona,AZ,7378494,6526.0,4868.0,671.0,1037.0,308.0,200.0,1383.0,275.0,1037.0,1.9,14019.1386
4,2020-04-26,California,CA,39937489,42164.0,NaN,4928.0,NaN,1473.0,NaN,NaN,1710.0,NaN,1.8,71887.4802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2722,2020-01-26,Washington,WA,7797095,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.7,13255.0615
2723,2020-01-25,Washington,WA,7797095,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.7,13255.0615
2724,2020-01-24,Washington,WA,7797095,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.7,13255.0615
2725,2020-01-23,Washington,WA,7797095,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.7,13255.0615


## Step 2: Some Exploratory Data Analysis (EDA)

In [17]:
# TODO Get some insights on data

## Step 3: Build model for missing data (maybe? Up for discussion)

In [18]:
## TODO Prepare necessary data for model (filter out with values which have all data)

## Step 4: Using findings from dataset of hospital beds, conclude research problem

Load and clean JHU data

In [116]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files('benhamner/jhucovid19', path='./kaggle/input/jhucovid19/', unzip=True)

# Get Time-Series Data of cases as Pandas DataFrame
dir_jhu = './kaggle/input/jhucovid19/csse_covid_19_data/csse_covid_19_daily_reports'

df_list = []
for dirname, _, files in os.walk(dir_jhu):
    for file in files:
        if 'gitignore' not in file and 'README' not in file:
            full_dir = os.path.join(dirname, file)
            #print(full_dir)
            df_list.append(pd.read_csv(full_dir))
jhu_df = pd.concat(df_list, axis=0, ignore_index=True, sort=True)

# combine Last Update with Last_Update
jhu_df['LastUpdate'] = jhu_df['Last_Update'].combine_first(jhu_df['Last Update'])

# format dates with parser
for i, row in jhu_df.iterrows():
    # update dates
    jhu_df.at[i, 'LastUpdate'] = parse(jhu_df.at[i, 'LastUpdate']).date()

# combine Country/Region with Country_Region
jhu_df['CountryRegion'] = jhu_df['Country/Region'].combine_first(jhu_df['Country_Region'])

# Retrieve only US data
jhu_df = jhu_df[jhu_df['CountryRegion']=='US']

# combine Province/State with Province_State
jhu_df['ProvinceState'] = jhu_df['Province/State'].combine_first(jhu_df['Province_State'])

# drop unnecessary columns
jhu_df = jhu_df.drop(['Admin2', 'Lat', 'Latitude', 'Long_', 'Longitude', 'Combined_Key', 'Country/Region',
                      'Country_Region', 'Province/State', 'Province_State',
                      'Last Update', 'Last_Update', 'FIPS'], axis=1)

# change column order
cols = list(jhu_df)
cols.insert(0, cols.pop(cols.index('CountryRegion')))
cols.insert(1, cols.pop(cols.index('ProvinceState')))
cols.insert(2, cols.pop(cols.index('LastUpdate')))
jhu_df = jhu_df.loc[:, cols]

# Change region to known US states
import us
state_abbrs = []
for state in us.states.STATES:
    state_abbrs.append(state.abbr)

for i, row in jhu_df.iterrows():
    for j in state_abbrs:
        if j in jhu_df.at[i, 'ProvinceState']:
            jhu_df.at[i, 'ProvinceState'] = us.states.lookup(j).name
        if jhu_df.at[i, 'ProvinceState'] == 'Washington, D.C.':
            jhu_df.at[i, 'ProvinceState'] = 'District of Columbia'

# Filter out unknown states
jhu_df = jhu_df[jhu_df['ProvinceState'].isin(list(all_cases.state.unique()))]

# Merge-sum rows with same date and State
jhu_df = jhu_df.groupby(['LastUpdate', 'ProvinceState']).agg(
    {
        'Active': sum,
        'Confirmed': sum,
        'Deaths': sum,
        'Recovered': sum
    }
).reset_index()

jhu_df

,LastUpdate,ProvinceState,Active,Confirmed,Deaths,Recovered
0,2020-01-22,Washington,0.0,1.0,0.0,0.0
1,2020-01-23,Washington,0.0,1.0,0.0,0.0
2,2020-01-24,Washington,0.0,1.0,0.0,0.0
3,2020-01-25,Illinois,0.0,1.0,0.0,0.0
4,2020-01-25,Washington,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
2493,2020-04-27,Vermont,805.0,851.0,46.0,0.0
2494,2020-04-27,Virginia,12521.0,12970.0,449.0,0.0
2495,2020-04-27,West Virginia,1021.0,1055.0,34.0,0.0
2496,2020-04-27,Wisconsin,5640.0,5912.0,272.0,0.0
